In [2]:
import pandas as pd

# Задание 1

Для датафрейма log из материалов занятия создайте столбец source_type по следующим правилам:

- если источник traffic_source равен yandex или google, то в source_type ставится organic  
- для источников paid и email из России - ставим ad  
- для источников paid и email не из России - ставим other  
- все остальные варианты берем из traffic_source без изменений  

In [16]:
%%time

def check_source(row):
    if row["traffic_source"] == "yandex" or row["traffic_source"] == "google":
        return "organic"
    elif (row["traffic_source"] == "paid" or row["traffic_source"] == "email") and row["region"]=="Russia":
        return "ad"
    elif (row["traffic_source"] == "paid" or row["traffic_source"] == "email") and row["region"]!="Russia":
        return "other"
    else:
        return row["traffic_source"]
        

log_df = pd.read_csv("dataset_lab_10/visit_log.csv", encoding = "utf-8", sep = ";")
log_df["source_type"] = log_df.apply(check_source, axis=1)
log_df

Wall time: 647 ms


,timestamp,visit_id,url,region,user_id,traffic_source,source_type
0,1549980692,e3b0c44298,https://host.ru/3c19b4ef7371864fa3,Russia,b1613cc09f,yandex,organic
1,1549980704,6e340b9cff,https://host.ru/c8d9213a31839f9a3a,Russia,4c3ec14bee,direct,direct
2,1549980715,96a296d224,https://host.ru/b8b58337d272ee7b15,Russia,a8c40697fb,yandex,organic
3,1549980725,709e80c884,https://host.ru/b8b58337d272ee7b15,Russia,521ac1d6a0,yandex,organic
4,1549980736,df3f619804,https://host.ru/b8b58337d272ee7b15,Russia,d7323c571c,yandex,organic
...,...,...,...,...,...,...,...
18933,1550094288,57e5ba8560,https://host.ru/c2382eb3d6afc8d0f3,Belarus,98b19810d0,paid,other
18934,1550094296,6f9389ec1b,https://host.ru/f1eb4601740d627ab0,Russia,32ebb20c13,paid,ad
18935,1550094308,e8cf2eb8e6,https://host.ru/a5dda93e70318570c0,Belarus,b85baa8c73,yandex,organic
18936,1550094314,79530b9a67,https://host.ru/6fda01ec57f23abc9e,Russia,e154b06121,paid,ad


# Задание 2

В файле URLs.txt содержатся url страниц новостного сайта. Вам необходимо отфильтровать его по адресам страниц с текстами новостей. Известно, что шаблон страницы новостей имеет внутри url следующую конструкцию: /, затем 8 цифр, затем дефис. Выполните следующие действия:

1. Прочитайте содержимое файла с датафрейм
2. Отфильтруйте страницы с текстом новостей, используя метод str.contains и регулярное выражение в соответствии с заданным шаблоном

In [25]:
url_df = pd.read_csv("dataset_lab_10/URLs.txt", encoding="utf-8", sep = "\n")
url_df[url_df.url.str.contains("/\d{8}-")]

,url
3,/politics/36188461-s-marta-zhizn-rossiyan-susc...
4,/world/36007585-tramp-pridumal-kak-reshit-ukra...
5,/science/36157853-nasa-sobiraet-ekstrennuyu-pr...
6,/video/36001498-poyavilis-pervye-podrobnosti-g...
7,/world/36007585-tramp-pridumal-kak-reshit-ukra...
...,...
88,/cis/35984145-kreml-prokommentiroval-soobschen...
89,/video/36071019-olimpiyskie-obekty-rio-prevrat...
90,/science/36151301-nazvano-posledstvie-zloupotr...
91,/incidents/36027330-vospitatelnitsu-zatravili-...


# Задание 3

Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId.

In [4]:
%%time
ratings_df = pd.read_csv("dataset_lab_10/ml-latest-small/ratings.csv", encoding="utf-8")
ratings_df.head()
test_df = ratings_df.groupby("userId").agg({"timestamp": ["min", "max"], "userId": "count"})
test_df["diff"] = test_df["timestamp"]["max"] - test_df["timestamp"]["min"]
test_df[test_df["userId"]["count"]>100]["diff"].mean()

Wall time: 154 ms


40080507.4496124

# Задание 4.1 (из python_pandas_pivot_tables)

В датафрейме data создайте столбец lemmas, в котором вычислите леммы поисковых запросов из столбца keyword. Леммы должны иметь строковый тип.

In [9]:
from pymystem3 import Mystem

def getting_lemmas(col):
    m = Mystem()
    return " ".join(m.lemmatize(col))

data = pd.DataFrame({
    'keyword': ['курс гривны к рублю', 'доллары в рубли', '100 долларов в рублях', 'курс рубля'],
    'shows': [125076, 114173, 97534, 53546],
})

data["lemmas"] = data.keyword.apply(getting_lemmas)
data

,keyword,shows,lemmas
0,курс гривны к рублю,125076,курс гривна к рубль \n
1,доллары в рубли,114173,доллар в рубль \n
2,100 долларов в рублях,97534,100 доллар в рубль \n
3,курс рубля,53546,курс рубль \n


# Задача 4.2 (из файла Python_13_join)

Дана статистика услуг перевозок клиентов компании по типам:

- rzd - железнодорожные перевозки  
- auto - автомобильные перевозки  
- air - воздушные перевозки  
- client_base - адреса клиентов

Необходимо сформировать две таблицы:
1. таблицу с тремя типами выручки для каждого client_id без указания адреса клиента
2. аналогичную таблицу по типам выручки с указанием адреса клиента  

Обратите внимание, что в процессе объединения таблиц данные не должны теряться.

In [4]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)

auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)

air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)

client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)

In [25]:
resulting_df_1 = pd.merge(client_base, rzd, on = "client_id", how = "outer").merge(auto, on = "client_id", how = "outer").merge(air, on = "client_id", how = "outer")
resulting_df_1

,client_id,address,rzd_revenue,auto_revenue,air_revenue
0,111,Комсомольская 4,1093.0,NaN,NaN
1,112,Энтузиастов 8а,2810.0,NaN,NaN
2,113,Левобережная 1а,10283.0,57483.0,NaN
3,114,Мира 14,5774.0,83.0,NaN
4,115,ЗЖБИиДК 1,981.0,912.0,81.0
5,116,Строителей 18,NaN,4834.0,4.0
6,117,Панфиловская 33,NaN,98.0,13.0
7,118,Мастеркова 4,NaN,NaN,173.0


In [26]:
resulting_df_2 = resulting_df_1.loc[:, resulting_df.columns!="address"].fillna(0)
resulting_df_2

,client_id,rzd_revenue,auto_revenue,air_revenue
0,111,1093.0,0.0,0.0
1,112,2810.0,0.0,0.0
2,113,10283.0,57483.0,0.0
3,114,5774.0,83.0,0.0
4,115,981.0,912.0,81.0
5,116,0.0,4834.0,4.0
6,117,0.0,98.0,13.0
7,118,0.0,0.0,173.0
